<h1>Get Data</h1>

<h2>Data Collection</h2>

Updated on a quarterly basis, this BigQuery dataset includes an archive of Stack Overflow content, including posts, votes, tags, and badges. This dataset is updated to mirror the Stack Overflow content on the Internet Archive.
More info about the dataset is given at: https://www.kaggle.com/stackoverflow/stackoverflow

To collect the data we need to gather Questions and Answer that were posted on Stack Overflow. Thus what we need are the following:

- Title
- Question body
- Answers for that question
- Votes for each answers

We decide to restrict the query only to questions that has 'python' has a tag, due to the abundance of Q&A in Stack Overflow, to perform better test and try to give more precise answers.
However this process can be done over other argument just by changing the LIKE '%python%' word.

In [1]:
'''
import bq_helper
import pyarrow
from bq_helper import BigQueryHelper
import os

#Initialize credential for google query
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="CREDENTIALS.json"
bq_assistant = BigQueryHelper("bigquery-public-data", "stackoverflow")

#Query for retrieve data
QUERY = "SELECT q.id, q.title, q.body, q.tags, a.body as answers, a.score FROM `bigquery-public-data.stackoverflow.posts_questions` AS q INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a ON q.id = a.parent_id WHERE q.tags LIKE '%python%' LIMIT 500000"
df = bq_assistant.query_to_pandas(QUERY)
df.to_csv('DB/Original_data.csv')
'''

'\nimport bq_helper\nimport pyarrow\nfrom bq_helper import BigQueryHelper\nimport os\n\n#Initialize credential for google query\nos.environ["GOOGLE_APPLICATION_CREDENTIALS"]="CREDENTIALS.json"\nbq_assistant = BigQueryHelper("bigquery-public-data", "stackoverflow")\n\n#Query for retrieve data\nQUERY = "SELECT q.id, q.title, q.body, q.tags, a.body as answers, a.score FROM `bigquery-public-data.stackoverflow.posts_questions` AS q INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a ON q.id = a.parent_id WHERE q.tags LIKE \'%python%\' LIMIT 500000"\ndf = bq_assistant.query_to_pandas(QUERY)\ndf.to_csv(\'DB/Original_data.csv\')\n'

In [2]:
import pandas as pd
import numpy as np
import preprocessing
from csv import reader 
import tfidf
import scipy

pd.reset_option("^display")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\andri\.virtualenvs\Progetto_StackOverflow-UQruahqZ\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_10624/836472124.py", line 1, in <module>
    import pandas as pd
  File "C:\Users\andri\.virtualenvs\Progetto_StackOverflow-UQruahqZ\lib\site-packages\pandas\__init__.py", line 22, in <module>
    from pandas.compat import (
  File "C:\Users\andri\.virtualenvs\Progetto_StackOverflow-UQruahqZ\lib\site-packages\pandas\compat\__init__.py", line 15, in <module>
    from pandas.compat.numpy import (
  File "C:\Users\andri\.virtualenvs\Progetto_StackOverflow-UQruahqZ\lib\site-packages\pandas\compat\numpy\__init__.py", line 7, in <module>
    from pandas.util.version import Version
  File "C:\Users\andri\.virtualenvs\Progetto_StackOverflow-UQruahqZ\lib\site-packages\pandas\util\__init__.py", line 

TypeError: object of type 'NoneType' has no len()

<h2>Load Data</h2>

In [ ]:
df = pd.read_csv('DB/Original_data.csv', index_col=[0])

#df = df.sample(frac = 0.2)

print('Missing Values:')
df.isna().sum()

<h2>Manipolate dataframe</h2>

In [ ]:
#create column with answers aggregate by title of the questions
df2 = df 
func = lambda x: "\n".join(x)
df2 = df2.groupby('id')["answers"].agg([("answers",func)])



In [ ]:
# concat answers for each post beacause there are a record with question duplicated for each answer to that question 


grouped = df.groupby(['id','title', 'body','tags'],as_index=False).agg("sum","score")

grouped_df = pd.DataFrame(grouped)
grouped_df = pd.merge(grouped_df, df2, left_on='id', right_on='id', how='left')
grouped_df



<h2>Preprocessing Part</h2>

In the preprocessing process here before we calling a function from the preprocessing file that remove the html tags and the part of text to not consider (for example the codes sections) and then we using other function to clean the text with NLP technics.

<h3>Manipolating answers</h3>

In [ ]:
#Removing tags
answers = grouped_df["answers"]
preprocessing.remove_tags(answers)

#Clearing text 
answers_processed = answers.apply(lambda x: preprocessing.clear_text(x))


In [ ]:
answers_processed.head()
answers_processed.isna().sum() 
grouped_df['answers_processed'] = answers_processed

<h3>Manipolating questions</h3>

In [ ]:
#Merge title with body 
questions = grouped_df["body"]
preprocessing.remove_tags(questions)
questions

In [ ]:
#Clearing text 
questions_processed = questions.apply(lambda x: preprocessing.clear_text(x))
grouped_df['questions_processed'] = questions_processed
questions_processed

<h3>Manipolating titles</h3>
Create a column only for the processed title of the questions 

In [ ]:
processed_title = grouped_df.title.apply(lambda x: preprocessing.clear_text(x))
grouped_df['processed_title'] = processed_title
processed_title 

Drop columns that are not utils anymore

In [ ]:
#post_corpus = processed_title + '\n '+ questions_processed + '\n ' + answers_processed
grouped_df.drop("answers", axis=1, inplace=True)
grouped_df.drop("body", axis=1, inplace=True)
#grouped_df["post_corpus"] = post_corpus
grouped_df["questions"] = questions
grouped_df

<h3>Filter Tags</h3>
Filter out the tags, selecting only the 30 most common for better processing, so we can have less variability in the data.

In [ ]:
# Convert raw text data of tags into lists
grouped_df["tags"] = grouped_df["tags"].apply(lambda x: x.split('|'))   

# Make a dictionary to count the frequencies for all tags
tag_freq_dict = {}

for tags in grouped_df["tags"]:
    for tag in tags:
        #Remove tags python, python2.7 e python3 for further processing 
        if "python" not in tag :
            
            if tag not in tag_freq_dict:
                tag_freq_dict[tag] = 0
            else:
                tag_freq_dict[tag] += 1
        else:
            tags.remove(tag)
            
grouped_df["tags"]


In [ ]:
#Selecting the most common number of tags in our database 
import heapq
most_common_tags = heapq.nlargest(30, tag_freq_dict, key=tag_freq_dict.get)
most_common_tags

Select only the data with at least one of the most common tags

In [ ]:
final_indices = []
for i,tags in enumerate(grouped_df["tags"].values.tolist()):
    if len(set(tags).intersection(set(most_common_tags)))>0:   # The minimum length for common tags should be 1
        final_indices.append(i)

final_data = grouped_df.iloc[final_indices]

final_data 

In [ ]:
# Normalize numeric data for the scores
final_data['score'] = (final_data['score'] - final_data['score'].mean()) / (final_data['score'].max() - final_data['score'].min())

In [ ]:
# Combine the lists back into text data
final_data['tags'] = final_data['tags'].apply(lambda x: '|'.join(x))

final_data

In [ ]:
#Check if the final data has some null values 
final_data.isna().sum()

final_data = final_data.dropna()

final_data = final_data[final_data['processed_title'].notna()]
final_data 

Eliminate null values if presents

In [ ]:
final_data = final_data[final_data['processed_title'].notna()]

final_data = final_data[final_data['questions_processed'].notna()]

final_data = final_data[final_data['answers_processed'].notna()]

In [ ]:
final_data['post_corpus'] = final_data['processed_title'] + final_data['questions_processed'] + final_data['answers_processed']

In [ ]:
# Save the data
final_data.to_csv('DB/Preprocessed_data.csv', index=False)